In [1]:
from flax import linen as nn
from flax.training import train_state
import jax
import jax.numpy as jnp
import numpy as np
import optax
import tensorflow_datasets as tfds
import tensorflow as tf 

from  stocks_dataset import x_train,x_test,y_train,y_test

In [ ]:
batch_size=100
num_epochs=30
class SimpleScan(nn.Module):
    @nn.compact
    def __call__(self, xs,is_training:bool=True):
        LSTM1 = nn.scan(nn.LSTMCell,
                    variable_broadcast="params",
                    split_rngs={"params": False},
                    in_axes=1,
                    out_axes=1)
                    
        c1 = nn.LSTMCell.initialize_carry(jax.random.PRNGKey(0), (batch_size,), 100)
        c1, xs=LSTM1()(c1, xs)
        # print(xs.shape)
        # print(xs.shape)
        return xs

In [5]:
import flax
import flax.linen as nn
from jax import random

class SimpleScan(nn.Module):
      @nn.compact
      def __call__(self, c, xs):
        LSTM = nn.scan(nn.LSTMCell,
                       variable_broadcast="params",
                       split_rngs={"params": False},
                       in_axes=1,
                       out_axes=1)
        return LSTM()(c, xs)

seq_len, batch_size, in_feat, out_feat = 20, 16, 30, 50
key_1, key_2, key_3 = random.split(random.PRNGKey(0), 3)

xs = random.uniform(key_1, (batch_size, seq_len, in_feat))
init_carry = nn.LSTMCell.initialize_carry(key_2, (batch_size,), out_feat)

model = SimpleScan()
variables = model.init(key_3, init_carry, xs)
out_carry, out_val = model.apply(variables, init_carry, xs)
print(jax.tree_util.tree_map(lambda x:x.shape,init_carry))
print(jax.tree_util.tree_map(lambda x:x.shape,variables))


((16, 50), (16, 50))
FrozenDict({
    params: {
        ScanLSTMCell_0: {
            hf: {
                bias: (50,),
                kernel: (50, 50),
            },
            hg: {
                bias: (50,),
                kernel: (50, 50),
            },
            hi: {
                bias: (50,),
                kernel: (50, 50),
            },
            ho: {
                bias: (50,),
                kernel: (50, 50),
            },
            if: {
                kernel: (30, 50),
            },
            ig: {
                kernel: (30, 50),
            },
            ii: {
                kernel: (30, 50),
            },
            io: {
                kernel: (30, 50),
            },
        },
    },
})
